## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Apr. 6th Apr. 13th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,April-9-2025,Condo/Co-op,1211 Meridian Ave #1,Miami Beach,FL,33139.0,345000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1211-Mer...,MARMLS,A11760393,N,Y,25.783112,-80.135972


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-04-07    29
2025-04-08    24
2025-04-10    22
2025-04-11    21
2025-04-09    20
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
102,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511
63,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11500630,N,Y,25.775127,-80.141257
69,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11497691,N,Y,25.775127,-80.141257
99,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653769,N,Y,25.941865,-80.120490
117,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11541481,N,Y,25.760000,-80.189163
110,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11603296,N,Y,25.802013,-80.185776
39,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11737396,N,Y,25.764264,-80.192725
5,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11757940,N,Y,25.922834,-80.121864
116,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #1705,Miami Beach,FL,33139.0,2735000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11348818,N,Y,25.774730,-80.141017
101,PAST SALE,2025-04-11,Condo/Co-op,2935 SW 30th Ct Unit B,Miami,FL,33133.0,2400000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2935-SW-30th-C...,MARMLS,A11651420,N,Y,25.736396,-80.242368


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,...,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None
1,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,None,None
2,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,None,None
3,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,...,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None
4,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,...,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Jeffrey Amoretti,Fortune Christie's International Real Estate,Jordan Lederman,Fortune International Realty,None,None,None,None
5,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,...,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Jordan Lederman,Fortune International Realty,None,None,MariaCecilia Hinojosa,The Corcoran Group,None,None
6,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,...,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Amanda Malone,Serhant,None,None
7,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,...,-80.121864,https://www.redfin.com/FL/North-Miami-Beach/15...,Lauren Sencion,Keller Williams Eagle Realty,None,None,3009174 Romanello,"Compass Florida, LLC.",None,None
8,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #1705,Miami Beach,FL,33139.0,2735000.0,2.0,2.5,...,-80.141017,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,None,None,Felipe Munoz,Million Realty LLC,None,None
9,PAST SALE,2025-04-11,Condo/Co-op,2935 SW 30th Ct Unit B,Miami,FL,33133.0,2400000.0,4.0,3.5,...,-80.242368,https://www.redfin.com/FL/Miami/2935-SW-30th-C...,Jessica Adams,Douglas Elliman,None,None,RICHARD Vazquez,Address Real Estate LLC,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

116


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,062,048


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$636


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$123,197,559


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
152
-----------
Input Previous Week Condo Average Sales Price:
867375
-----------
Input Previous Week Condo Average PSF:
585
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
131841006.0


In [26]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,152,867375,585,131841006.0


In [27]:
last_week['Last Week Total Dollar Volume'].iloc[0]

131841006.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          116             1062048          636                    123197559.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange
1,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,FIVE PARK,3186.0,NaN,2024.0,NaN,2839.0,6400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11500630,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,Miami Beach,2615.0,NaN,2024.0,NaN,3147.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11497691,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,None,None,3,blue
3,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3287.0,NaN,2022.0,NaN,2282.0,7338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653769,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,4,blue
4,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,JADE RESIDENCES BRICKELL,3415.0,NaN,2004.0,NaN,1318.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11541481,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Jeffrey Amoretti,Fortune Christie's International Real Estate,Jordan Lederman,Fortune International Realty,None,None,None,None,5,blue
5,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,MISSONI BAIA,3788.0,NaN,2020.0,NaN,1063.0,7926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11603296,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Jordan Lederman,Fortune International Realty,None,None,MariaCecilia Hinojosa,The Corcoran Group,None,None,6,blue
6,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,BRICKELL FLATIRON CONDO,2342.0,NaN,2019.0,NaN,1403.0,3546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11737396,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Amanda Malone,Serhant,None,None,7,blue
7,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,TDR TOWER III CONDO,2928.0,NaN,2010.0,NaN,951.0,3510.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11757940,N,Y,25.922834,-80.121864,https://www.redfin.com/FL/North-Miami-Beach/15...,Lauren Sencion,Keller Williams Eagle Realty,None,None,3009174 Romanello,"Compass Florida, LLC.",None,None,8,blue
8,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #1705,Miami Beach,FL,33139.0,2735000.0,2.0,2.5,FIVE PARK,1516.0,NaN,2024.0,NaN,1804.0,2805.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500

In [36]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_04142025


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'OCEANSIDE @ FISHER ISL CO at 7964 Fisher Island Dr #7964 in Miami Beach'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange,OCEANSIDE @ FISHER ISL CO at 7964 Fisher Islan...
1,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,FIVE PARK,3186.0,NaN,2024.0,NaN,2839.0,6400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11500630,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,None,None,2,blue,FIVE PARK at 500 Alton #4401 in Miami Beach
2,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,Miami Beach,2615.0,NaN,2024.0,NaN,3147.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11497691,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,None,None,3,blue,Miami Beach at 500 Alton Rd #4403 in Miami Beach
3,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3287.0,NaN,2022.0,NaN,2282.0,7338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653769,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,4,blue,Estates at Acqualina at 17901 Collins Ave #210...
4,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,JADE RESIDENCES BRICKELL,3415.0,NaN,2004.0,NaN,1318.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11541481,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Jeffrey Amoretti,Fortune Christie's International Real Estate,Jordan Lederman,Fortune International Realty,None,None,None,None,5,blue,JADE RESIDENCES BRICKELL at 1331 Brickell Bay ...
5,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,MISSONI BAIA,3788.0,NaN,2020.0,NaN,1063.0,7926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11603296,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Jordan Lederman,Fortune International Realty,None,None,MariaCecilia Hinojosa,The Corcoran Group,None,None,6,blue,MISSONI BAIA at 700 NE 26 Ter #4003 in Miami
6,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,BRICKELL FLATIRON CONDO,2342.0,NaN,2019.0,NaN,1403.0,3546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11737396,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Amanda Malone,Serhant,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,TDR TOWER III CONDO,2928.0,NaN,2010.0,NaN,951.0,3510.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11757940,N,Y,25.922834,-80.121864,https:/

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANSIDE @ FISHER ISL CO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,400,000 to $20,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 116 condo sales totaling $123,197,559 million from Apr. 6th to Apr. 13th. The previous week, brokers closed 152 condo sales totaling $131,841,006.

Last week’s units sold for an average of $1,062,048, higher than the $867,375 average price from the pre

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange,OCEANSIDE @ FISHER ISL CO at 7964 Fisher Islan...
1,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,FIVE PARK,3186.0,NaN,2024.0,NaN,2839.0,6400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11500630,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,None,None,2,blue,FIVE PARK at 500 Alton #4401 in Miami Beach
2,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,Miami Beach,2615.0,NaN,2024.0,NaN,3147.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11497691,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,None,None,3,blue,Miami Beach at 500 Alton Rd #4403 in Miami Beach
3,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3287.0,NaN,2022.0,NaN,2282.0,7338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653769,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,4,blue,Estates at Acqualina at 17901 Collins Ave #210...
4,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,JADE RESIDENCES BRICKELL,3415.0,NaN,2004.0,NaN,1318.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11541481,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Jeffrey Amoretti,Fortune Christie's International Real Estate,Jordan Lederman,Fortune International Realty,None,None,None,None,5,blue,JADE RESIDENCES BRICKELL at 1331 Brickell Bay ...
5,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,MISSONI BAIA,3788.0,NaN,2020.0,NaN,1063.0,7926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11603296,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Jordan Lederman,Fortune International Realty,None,None,MariaCecilia Hinojosa,The Corcoran Group,None,None,6,blue,MISSONI BAIA at 700 NE 26 Ter #4003 in Miami
6,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,BRICKELL FLATIRON CONDO,2342.0,NaN,2019.0,NaN,1403.0,3546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11737396,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Amanda Malone,Serhant,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,TDR TOWER III CONDO,2928.0,NaN,2010.0,NaN,951.0,3510.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11757940,N,Y,25.922834,-80.121864,https:/

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: OCEANSIDE @ FISHER ISL CO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,400,000 to $20,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 116 condo sales totaling $123,197,559 million from Apr. 6th to Apr. 13th. The previous week, brokers closed 152 condo sales totaling $131,841,006.

Last week’s units sold for an average of $1,062,048, higher than the $867,375 average price from the pre

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/7964-Fisher-Island-Dr-33109/unit-7964/home/43094921


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4401/home/188922927


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/2935-SW-30th-Ct-33133/unit-B/home/42753482


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-1705/home/184549185


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4403/home/188851945


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/2935-SW-30th-Ct-33133/unit-B/home/42753482


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-07,Condo/Co-op,7964 Fisher Island Dr #7964,Miami Beach,FL,33109.0,20500000.0,5.0,7.5,OCEANSIDE @ FISHER ISL CO,6820.0,NaN,1989.0,NaN,3006.0,14131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,MARMLS,A11611096,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7964-Fis...,Karla Abaunza,Luxury Living Realty,Maria Oneide Gioia,Luxury Living Realty,Laura Castillo,The Corcoran Group,None,None,1,orange,OCEANSIDE @ FISHER ISL CO at 7964 Fisher Islan...
1,PAST SALE,2025-04-08,Condo/Co-op,500 Alton #4401,Miami Beach,FL,33139.0,9044660.0,4.0,4.5,FIVE PARK,3186.0,NaN,2024.0,NaN,2839.0,6400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11500630,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,None,None,2,blue,FIVE PARK at 500 Alton #4401 in Miami Beach
2,PAST SALE,2025-04-08,Condo/Co-op,500 Alton Rd #4403,Miami Beach,FL,33139.0,8230000.0,3.0,4.5,Miami Beach,2615.0,NaN,2024.0,NaN,3147.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11497691,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Charlotte Maietto,Douglas Elliman,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,None,None,3,blue,Miami Beach at 500 Alton Rd #4403 in Miami Beach
3,PAST SALE,2025-04-07,Condo/Co-op,17901 Collins Ave #2102,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3287.0,NaN,2022.0,NaN,2282.0,7338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653769,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,4,blue,Estates at Acqualina at 17901 Collins Ave #210...
4,PAST SALE,2025-04-11,Condo/Co-op,1331 Brickell Bay Dr #3211,Miami,FL,33131.0,4500000.0,4.0,4.5,JADE RESIDENCES BRICKELL,3415.0,NaN,2004.0,NaN,1318.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11541481,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Jeffrey Amoretti,Fortune Christie's International Real Estate,Jordan Lederman,Fortune International Realty,None,None,None,None,5,blue,JADE RESIDENCES BRICKELL at 1331 Brickell Bay ...
5,PAST SALE,2025-04-11,Condo/Co-op,700 NE 26 Ter #4003,Miami,FL,33137.0,4025000.0,4.0,6.0,MISSONI BAIA,3788.0,NaN,2020.0,NaN,1063.0,7926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11603296,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Jordan Lederman,Fortune International Realty,None,None,MariaCecilia Hinojosa,The Corcoran Group,None,None,6,blue,MISSONI BAIA at 700 NE 26 Ter #4003 in Miami
6,PAST SALE,2025-04-09,Condo/Co-op,1000 Brickell Plz Ph 5002,Miami,FL,33131.0,3286000.0,3.0,3.5,BRICKELL FLATIRON CONDO,2342.0,NaN,2019.0,NaN,1403.0,3546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11737396,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Amanda Malone,Serhant,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-04-07,Condo/Co-op,15811 Collins Ave #3601,Sunny Isles Beach,FL,33160.0,2785000.0,3.0,3.5,TDR TOWER III CONDO,2928.0,NaN,2010.0,NaN,951.0,3510.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11757940,N,Y,25.922834,-80.121864,https:/

## Time on Market Calculator

In [63]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 12, 13) ## List (Earlier) date
date2 = datetime(2025, 4, 8) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

482


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df['$/SQUARE FEET'].astype(float)

0    3006.0
1    2839.0
2    3147.0
3    2282.0
4    1318.0
5    1063.0
6    1403.0
7     951.0
8    1804.0
9     817.0
Name: $/SQUARE FEET, dtype: float64

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-04-07,7964 Fisher Island Dr #7964 in Miami Beach,"$20,500,000",5,7.5,"6,820",1989,"$3,006",MARMLS,A11611096,Karla Abaunza Luxury Living Realty Maria Oneid...,Laura Castillo The Corcoran Group
1,2025-04-08,500 Alton #4401 in Miami Beach,"$9,044,660",4,4.5,"3,186",2024,"$2,839",MARMLS,A11500630,Anca Mirescu Douglas Elliman Charlotte Maietto...,Anca Mirescu Douglas Elliman
2,2025-04-08,500 Alton Rd #4403 in Miami Beach,"$8,230,000",3,4.5,"2,615",2024,"$3,147",MARMLS,A11497691,Charlotte Maietto Douglas Elliman Anca Mirescu...,Charlotte Maietto Douglas Elliman
3,2025-04-07,17901 Collins Ave #2102 in Sunny Isles Beach,"$7,500,000",4,5.0,"3,287",2022,"$2,282",MARMLS,A11653769,Ryan Mendell Maxwell E Realty Inc,Yansy Checa Douglas Elliman
4,2025-04-11,1331 Brickell Bay Dr #3211 in Miami,"$4,500,000",4,4.5,"3,415",2004,"$1,318",MARMLS,A11541481,Jeffrey Amoretti Fortune Christie's Internatio...,
5,2025-04-11,700 NE 26 Ter #4003 in Miami,"$4,025,000",4,6.0,"3,788",2020,"$1,063",MARMLS,A11603296,Jordan Lederman Fortune International Realty,MariaCecilia Hinojosa The Corcoran Group
6,2025-04-09,1000 Brickell Plz Ph 5002 in Miami,"$3,286,000",3,3.5,"2,342",2019,"$1,403",MARMLS,A11737396,"Diego Valencia Compass Florida, LLC",Amanda Malone Serhant
7,2025-04-07,15811 Collins Ave #3601 in Sunny Isles Beach,"$2,785,000",3,3.5,"2,928",2010,$951,MARMLS,A11757940,Lauren Sencion Keller Williams Eagle Realty,"3009174 Romanello Compass Florida, LLC."
8,2025-04-08,500 Alton #1705 in Miami Beach,"$2,735,000",2,2.5,"1,516",2024,"$1,804",MARMLS,A11348818,Anca Mirescu Douglas Elliman,Felipe Munoz Million Realty LLC
9,2025-04-11,2935 SW 30th Ct Unit B in Miami,"$2,400,000",4,3.5,"2,936",2019,$817,MARMLS,A11651420,Jessica Adams Douglas Elliman,RICHARD Vazquez Address Real Estate LLC
